In [3]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import pathlib
import sys
import datetime as dt
import lightgbm as lgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import pickle

ROOT_PATH = pathlib.Path().resolve().parent
ROOT_PATH

sys.path.append(str(ROOT_PATH))

from modules.date_range import date_range

In [1]:
import xarray as xr
import pathlib
import sys
import datetime as dt
import lightgbm as lgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import pickle

ROOT_PATH = pathlib.Path().resolve().parent
ROOT_PATH

sys.path.append(str(ROOT_PATH))

from modules.date_range import date_range


start = dt.date(2022,1,1)
end = dt.date(2023,1,1)
datasets = [xr.open_dataset(date.strftime(f"{ROOT_PATH}/data/%Y/%m/%Y_%m%d.nc"))[['psea', 'sp', 'u', 'v', 'temp', 'rh', 'r1h', 'dswrf','ncld']].dropna(dim="time") for date in date_range(start,end)]
print("dataset Success")
del start,end


combined_dataset = xr.concat(datasets,dim="time")
print("combine Success")

del datasets


# 特徴量とターゲット変数に分割する
X = combined_dataset[['psea', 'sp', 'u', 'v', 'temp', 'rh', 'r1h', 'dswrf']]
y = combined_dataset['ncld']

del combined_dataset


X_df = X.to_dataframe()
print("X_df Success")
del X


y_df = y.to_dataframe()
print("y_df Success")
del y


# データセットをトレーニングセットとテストセットに分割する
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.3)

del X_df,y_df
del X_train,y_train

dataset Success
combine Success
X_df Success
y_df Success


In [11]:
X_test.reset_index(inplace=True)

In [13]:
y_test = y_test.reset_index()["ncld"]

In [12]:
### csvから
X_df = pd.read_csv(ROOT_PATH/"csv/2023-07-03X_df.csv")
y_df = pd.read_csv(ROOT_PATH/"csv/2023-07-03y_df.csv")["ncld"]

In [13]:
X_train,X_test,y_train,y_test = train_test_split(X_df,y_df,test_size=0.3,random_state=1)

In [14]:
X_test_drop = X_test.drop(["time","lat","lon"],axis=1)

In [15]:
with open(ROOT_PATH/"models/model_using_msm_extract_only_amedas_point.pkl", 'rb') as f:
    model = pickle.load(f)

In [16]:
X_test_drop

,psea,sp,u,v,temp,rh,r1h,dswrf
0,100855.504480,100851.375764,-4.428135,0.324159,280.375021,94.245412,200.000000,665.0000
1,101440.825570,101440.366578,-2.617737,10.458716,300.569469,85.403669,0.544336,73.2265
2,102491.284266,102489.907844,-7.327217,-1.107034,290.533662,78.552752,-0.000007,46.4945
3,100438.532010,100435.779441,10.697248,2.581040,276.232637,68.477065,-0.000007,0.0005
4,102274.770509,102275.228948,-1.694190,-3.859327,282.029361,61.247707,-0.000007,0.0005
...,...,...,...,...,...,...,...,...
638354335,101050.458604,101055.962916,3.510703,4.446483,298.361069,91.165137,200.000000,665.0000
638354336,102675.687932,97610.091419,2.244648,0.232416,262.681684,40.800460,-0.000007,512.2135
638354337,101404.128323,101409.173918,2.752294,-4.171254,280.435131,77.309633,200.000000,665.0000
638354338,100717.431088,100713.761087,-2.935780,4.232416,293.763937,81.621559,-0.000007,419.1435


In [17]:
y_test

0             6.006669
1            56.885000
2            46.875000
3             0.000002
4            35.595001
               ...    
638354335    22.606668
638354336     1.075002
638354337     1.025002
638354338    52.246667
638354339    38.183334
Name: ncld, Length: 638354340, dtype: float64

In [18]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

y_pred = model.predict(X_test_drop)

In [46]:
y_test

357457     55.126666
127869     28.320001
1358895    61.034999
579704      2.001669
567645      9.716668
             ...    
131374      0.928336
1571181     3.856669
1040615    38.818334
1609801     1.855002
1037549    20.751668
Name: ncld, Length: 596556, dtype: float64

In [19]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

rmse = np.sqrt(mean_squared_error(y_test.values,y_pred))
r2 = r2_score(y_test.values,y_pred)

In [20]:
print(f"RMSE: {rmse}")
print(f"R2: {r2}")

RMSE: 21.197622583619264
R2: 0.5653030738663836


In [17]:
print(f"RMSE: {rmse}")
print(f"R2: {r2}")

RMSE: 21.228720399427075
R2: 0.5636176913510266


In [47]:
ans = X_test.copy()
ans["predict_ncld"] = y_pred
ans["correct_ncld"] = y_test
ans["ncld_diff"] = y_test - y_pred

In [49]:
ans.to_csv(ROOT_PATH/"csv/ans_use_msm_extract_anedas_point.csv",index=False)